# Extract data for meta-analysis

In [4]:
from ollama import Client
from langchain_community.document_loaders import PyPDFLoader
from ollama import Options

In [6]:
pdf_path = '/Users/jk1/Downloads/120-neurological-intracranial_pressure-Aletreby-2021.pdf' 

In [19]:
client = Client(
  host='http://localhost:11434',
  headers={'x-some-header': 'some-value'}
)

In [10]:
MAIN_TASK = """
A) Est-ce que l'étude mise en pièce jointe indique (merci de répondre dans cet ordre, par des mots clés, en anglais) :
- le type de population dont il s'agit 
- le nombre total de patients inclus
- le setting dans lequel ont eu lieu la majorité des études (urgences, soins intensifs) 
- le nombre total d'études 
- le symptôme qui est investigué (motif de consultation), 
- quel est le diagnostic recherché
- quel outil diagnostique est utilisé (quel type ou technique d'ultrasonographie)
- avec quel modalité diagnostique (imagerie ou autre gold standart utilisé) il est comparé.
Pour cette dernière partie de la réponse, merci de me ranger ces résultats dans un tableau excel avec une colonne par "tiret" sus-mentionné. Tu dois remplir les cases en vert si tu es confident et en jaune si tu ne l'es pas, et en rouge si tu ne sais pas

B) Concernant l'étude mise en pièce jointe, Indique moi pour le diagnostic étudié
- le nombre de fp	
-  le nombre de fn
-  le nombre de tp 
-  le nombre de tn
- la prevalence 
- la limite basse de l'intervalle de confiance de la prevalence (en pourcentage)
- la limite haute de l'intervalle de confiance de la prevalence (en pourcentage)
- La sensibilité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- la limite basse de l'intervalle de la sensibilité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- la limite haute de l'intervalle de la sensibilité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- La valeur I2 de l'hétérogenéité de la sensibilité
- la valeur p de l'hétérogenéité de la sensibilité
- La spécificité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- la limite basse de l'intervalle de la spécificité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- la limite haute de l'intervalle de la spécificité moyenne (combinée, en nombre absolue de pourcentage avec 2 chiffre avant la figure) 
- La valeur I2 de l'hétérogenéité de la spécificité 
- la valeur p de l'hétérogenéité de la spécificité 
- la valeur de l'AUC
- l'intervalle de confiance de l'AUC
Pour cette dernière partie de la réponse, merci de me ranger ces résultats dans un tableau excel avec une colonne par "tiret" sus-mentionné (les item ci-dessus doivent représenter l'axe des ordonnées). Tu dois remplir les cases en vert si tu es confident et en jaune si tu ne l'es pas, et en rouge si tu ne sais pas

C) Est-ce que l'étude s’intéresse à plusieurs diagnostics ? Si oui, merci de recréer les deux tableaux demandés ci-dessus pour chaque diagnostic précis (en abcisses). Tu dois remplir les cases en vert si tu es confident et en jaune si tu ne l'es pas, et en rouge si tu ne sais pas.
"""

In [9]:
loader = PyPDFLoader(
    pdf_path,
    mode="single",
)
docs = loader.load()

In [25]:
len(docs[0].page_content) * 3

131850

In [36]:
def formulate_prompt(guideline_text, task):
    main_task = "Answer if the provided study indicates the following (only answer with keywords in English):\n"
    prompt = f"Task: {main_task}\n{task}\n\n Text:\n\n{guideline_text}\n\nProvide a short answer. Extracted data:\n"
    return prompt

In [37]:
task = "\nle type de population dont il s'agit"

response = client.generate(model='deepseek-r1:32b', prompt=formulate_prompt(docs[0].page_content, task),
                           options=Options(
                temperature=0.0,
                num_ctx=4096,
            ),)
response.response

"<think>\nOkay, so I need to figure out how to extract data from this list of references provided. The user has given me a bunch of citations, each with details like authors, journal names, years, and DOIs. They want a short answer based on the extracted data.\n\nFirst, I should understand what exactly they're asking for. It seems like they want key information pulled out from these references, maybe something specific like the main findings or important details about optic nerve sheath diameter (ONSD) measurements in relation to intracranial pressure (ICP). \n\nLooking at the references, many of them are studies that correlate ONSD with ICP. For example, reference 37 mentions a comparison between ultrasonographic measurement of ONSD and direct ICP measurement in TBI patients. That's probably important because it shows how accurate ONSD is as a non-invasive method.\n\nI should also note the methods used in these studies. Some are observational, like reference 34 which uses optic nerve 

In [38]:
print(response.response)

<think>
Okay, so I need to figure out how to extract data from this list of references provided. The user has given me a bunch of citations, each with details like authors, journal names, years, and DOIs. They want a short answer based on the extracted data.

First, I should understand what exactly they're asking for. It seems like they want key information pulled out from these references, maybe something specific like the main findings or important details about optic nerve sheath diameter (ONSD) measurements in relation to intracranial pressure (ICP). 

Looking at the references, many of them are studies that correlate ONSD with ICP. For example, reference 37 mentions a comparison between ultrasonographic measurement of ONSD and direct ICP measurement in TBI patients. That's probably important because it shows how accurate ONSD is as a non-invasive method.

I should also note the methods used in these studies. Some are observational, like reference 34 which uses optic nerve sonograp